In [ ]:
%pip install ragas

In [6]:
reference_data = [
  {
    "question": "How cancel subscription?", 
    "ground_truth": "Login and navigate to the account settings page to cancel your subscription.", #Expected llm generated answer
    "context": "Login and navigate to the account settings page to cancel your subscription. " #Expected retrieved context
  }
]
question = reference_data[0]['question']
ground_truth = reference_data[0]['ground_truth']
context = reference_data[0]['context']
print (f"question: {question}")
print (f"ground_truth: {ground_truth}")
print (f"context: {context}")

question: How cancel subscription?
ground_truth: Login and navigate to the account settings page to cancel your subscription.
context: Login and navigate to the account settings page to cancel your subscription. 


In [9]:
# Retrieve context from Milvus DB

from milvus_chatbot_with_rag import retrieve_similiar_contexts

def perform_retrieval(question):

    retrieved_context = retrieve_similiar_contexts(question, "policy_docs_collection1", 3)[0]['content']
    print (f"perform_retrieval.retrieved_context: {retrieved_context}")
    return retrieved_context
retrieved_context = perform_retrieval("How cancel subscription?")

Connected to Milvus on Zilliz Cloud
perform_retrieval.retrieved_context: Employees must submit a leave request for approval.


In [10]:
# Validate retrieved context against reference data    

print(f"Retrieved context: {retrieved_context}")
print(f"Expected context: {context}")
print(f"question: {question}")

Retrieved context: Employees must submit a leave request for approval.
Expected context: Login and navigate to the account settings page to cancel your subscription. 
question: How cancel subscription?


In [ ]:
# %pip install ragas datasets 

In [11]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_precision, context_recall

from dotenv import load_dotenv
from openai import OpenAI
import os

# --- Load API Key ---
load_dotenv(override=True, dotenv_path="../.env.local")
my_api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=my_api_key)

# question = reference_data[0]['question']
# ground_truth = reference_data[0]['ground_truth']
# context = reference_data[0]['context']
# Question User asked
question = reference_data[0]['question']

# Reference context (should be a string)
reference_context = reference_data[0]['context']

# Retrieved context (a string from perform_retrieval)
retrieved_context = [perform_retrieval(question)]

# Build dataset properly
dataset_dict = {
    "question": [question],
    "contexts": [retrieved_context],      # list of strings INSIDE another list
    "ground_truth": [reference_context],   # single string
    "answer": [""]
}

print(f"dataset_dict: {dataset_dict}")

ragas_dataset = Dataset.from_dict(dataset_dict)


# Evaluate retrieval
results = evaluate(
    dataset=ragas_dataset,
    metrics=[context_precision, context_recall]
)

C:\Users\sarat\AppData\Local\Temp\ipykernel_39652\1284853583.py:3: DeprecationWarning: Importing context_precision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_precision
  from ragas.metrics import context_precision, context_recall
C:\Users\sarat\AppData\Local\Temp\ipykernel_39652\1284853583.py:3: DeprecationWarning: Importing context_recall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_recall
  from ragas.metrics import context_precision, context_recall


Connected to Milvus on Zilliz Cloud
perform_retrieval.retrieved_context: Employees must submit a leave request for approval.
dataset_dict: {'question': ['How cancel subscription?'], 'contexts': [['Employees must submit a leave request for approval.']], 'ground_truth': ['Login and navigate to the account settings page to cancel your subscription. '], 'answer': ['']}


Evaluating: 100%|██████████| 2/2 [00:07<00:00,  3.98s/it]


In [12]:
print("Retrieval Evaluation Results:")
results.to_pandas()

Retrieval Evaluation Results:


,user_input,retrieved_contexts,response,reference,context_precision,context_recall
0,How cancel subscription?,[Employees must submit a leave request for app...,,Login and navigate to the account settings pag...,0.0,0.0
